In [12]:
import numpy as np
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Concatenate, Dropout, \
                        Flatten, MaxPooling2D, Multiply, Lambda, Add
from keras.backend import constant
from keras import optimizers

from keras.engine.topology import Layer
from keras.models import Model
from keras.layers import Input
from keras import initializers
from keras.constraints import max_norm
import keras.backend as K

from tensorflow import keras
import datetime

In [13]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

In [14]:
img_rows, img_cols = (28, 28)
train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, 1)
test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
half_validate = round(test_images.shape[0] / 2)

In [15]:
m = 20*20 # dimension of state space
d = 2 # number of controls
N = 40 # length of the sequence of increments
state = Input(shape=input_shape)
inputs = [state]
state = Conv2D(28, kernel_size=(3,3),trainable=False)(state)
state = Flatten()(state)
state = Dense(m, activation='tanh',trainable=False,
              kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
              bias_initializer='random_normal')(state)
output_state = []
layers = []
for j in range(N):
    for i in range(d):
        layer = Dense(m, activation='tanh',trainable=False,
                      kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
                      bias_initializer='random_normal',
                      name=str(i)+str(j))
        layers = layers + [layer]

for j in range(N):
    for i in range(d):
        #layer=Dense(m, activation='tanh',trainable=False,
        #            kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
        #            bias_initializer='random_normal',
        #            name=str(i)+str(j))
        Vx = layers[i+(j)*d](state)
        #Vx  = Dense(m, activation='tanh',trainable=False,
        #            kernel_initializer=initializers.RandomNormal(0,1),#kernel_initializer='random_normal',
        #            bias_initializer='random_normal')(state) # generation of the vector field
        #Vx  = Dense(m, activation='tanh',trainable=False,
        #            kernel_initializer='random_normal',
        #            bias_initializer='random_normal')(Vxhelper)
        input_control_incr = Input(shape=(1,))
        helper = Dense(1, activation='linear',trainable=True,
                       kernel_initializer=initializers.RandomNormal(1,0.),
                       bias_constraint=max_norm(0.),name='u'+str(i)+str(j))(input_control_incr)
        helperlist = [helper for i in range(m)]
        delta_u = Concatenate()(helperlist) # increments vectorized       
        mult = Multiply()([Vx, delta_u])
        state = Add()([mult,state])
        inputs = inputs + [input_control_incr]
    output_state=output_state+[state]
state = output_state[-1]
state = Dense(10, activation=tf.nn.softmax,trainable=True,
              kernel_initializer='random_uniform',
              bias_initializer='zeros')(state)
model = Model(inputs, outputs=state)

In [16]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_82 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 26, 26, 28)   280         input_82[0][0]                   
__________________________________________________________________________________________________
flatten_2 (Flatten)             (None, 18928)        0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
input_83 (InputLayer)           (None, 1)            0                                            
____________________________________________________________________________________________

                                                                 u11[0][0]                        
                                                                 u11[0][0]                        
                                                                 u11[0][0]                        
                                                                 u11[0][0]                        
                                                                 u11[0][0]                        
                                                                 u11[0][0]                        
                                                                 u11[0][0]                        
                                                                 u11[0][0]                        
                                                                 u11[0][0]                        
                                                                 u11[0][0]                        
          

                                                                 u03[0][0]                        
                                                                 u03[0][0]                        
                                                                 u03[0][0]                        
                                                                 u03[0][0]                        
                                                                 u03[0][0]                        
                                                                 u03[0][0]                        
                                                                 u03[0][0]                        
                                                                 u03[0][0]                        
                                                                 u03[0][0]                        
                                                                 u03[0][0]                        
          

                                                                 u05[0][0]                        
                                                                 u05[0][0]                        
                                                                 u05[0][0]                        
                                                                 u05[0][0]                        
                                                                 u05[0][0]                        
                                                                 u05[0][0]                        
                                                                 u05[0][0]                        
                                                                 u05[0][0]                        
                                                                 u05[0][0]                        
                                                                 u05[0][0]                        
          

                                                                 u16[0][0]                        
                                                                 u16[0][0]                        
                                                                 u16[0][0]                        
                                                                 u16[0][0]                        
                                                                 u16[0][0]                        
                                                                 u16[0][0]                        
                                                                 u16[0][0]                        
                                                                 u16[0][0]                        
                                                                 u16[0][0]                        
                                                                 u16[0][0]                        
          

...

u032 (Dense)                    (None, 1)            2           input_147[0][0]                  
__________________________________________________________________________________________________
032 (Dense)                     (None, 400)          160400      add_144[0][0]                    
__________________________________________________________________________________________________
concatenate_145 (Concatenate)   (None, 400)          0           u032[0][0]                       
                                                                 u032[0][0]                       
                                                                 u032[0][0]                       
                                                                 u032[0][0]                       
                                                                 u032[0][0]                       
                                                                 u032[0][0]                       
          

                                                                 u133[0][0]                       
                                                                 u133[0][0]                       
                                                                 u133[0][0]                       
                                                                 u133[0][0]                       
                                                                 u133[0][0]                       
                                                                 u133[0][0]                       
                                                                 u133[0][0]                       
                                                                 u133[0][0]                       
                                                                 u133[0][0]                       
                                                                 u133[0][0]                       
          

                                                                 u135[0][0]                       
                                                                 u135[0][0]                       
                                                                 u135[0][0]                       
                                                                 u135[0][0]                       
                                                                 u135[0][0]                       
                                                                 u135[0][0]                       
                                                                 u135[0][0]                       
                                                                 u135[0][0]                       
                                                                 u135[0][0]                       
                                                                 u135[0][0]                       
          

                                                                 u137[0][0]                       
                                                                 u137[0][0]                       
                                                                 u137[0][0]                       
                                                                 u137[0][0]                       
                                                                 u137[0][0]                       
                                                                 u137[0][0]                       
                                                                 u137[0][0]                       
                                                                 u137[0][0]                       
                                                                 u137[0][0]                       
                                                                 u137[0][0]                       
          

                                                                 u039[0][0]                       
                                                                 u039[0][0]                       
                                                                 u039[0][0]                       
                                                                 u039[0][0]                       
                                                                 u039[0][0]                       
                                                                 u039[0][0]                       
                                                                 u039[0][0]                       
                                                                 u039[0][0]                       
                                                                 u039[0][0]                       
                                                                 u039[0][0]                       
          

In [22]:
train_images.shape

(60000, 28, 28, 1)

In [8]:
# x_train_new = [x_train] + [1/N*np.ones((60000,1)) for i in range(N*d)]
x_train_new = [train_images] + [1/N*np.ones((60000,1)) for i in range(N*d)]
# x_test_new = [x_test] + [1/N*np.ones((10000,1)) for i in range(N*d)]
x_test_new = [test_images] + [1/N*np.ones((10000,1)) for i in range(N*d)]
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [10]:
# model.fit(x=x_train_new,y=y_train, epochs=1)
model.fit(x=x_train_new,y=train_labels, epochs=1)
# x = model.evaluate(x_test_new, y_test)
x = model.evaluate(x_test_new, test_labels)
print('\n',x)

Epoch 1/1
10000/10000 [==============================] - 15s 2ms/step

 [0.5030135055065155, 0.8180000185966492]


In [11]:
date_of_training = datetime.datetime.now().strftime("%Y%m%d-%H%M%S");
log_dir="logs/fit/" + "residual" + "/" + date_of_training
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(
    x_train_new,
    train_labels,
    epochs=100,
    shuffle=True,
    validation_data=(x_test_new, test_labels),
    callbacks=[
        keras.callbacks.EarlyStopping(
            patience=5,
            restore_best_weights=True,
        )
#         tensorboard_callback
    ]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 280s 5ms/step - loss: 0.5995 - accuracy: 0.7877 - val_loss: 0.5311 - val_accuracy: 0.8087
Epoch 2/100
60000/60000 [==============================] - 274s 5ms/step - loss: 0.4930 - accuracy: 0.8205 - val_loss: 0.5103 - val_accuracy: 0.8165
Epoch 3/100
60000/60000 [==============================] - 271s 5ms/step - loss: 0.4767 - accuracy: 0.8269 - val_loss: 0.5065 - val_accuracy: 0.8181
Epoch 4/100
60000/60000 [==============================] - 276s 5ms/step - loss: 0.4672 - accuracy: 0.8298 - val_loss: 0.5115 - val_accuracy: 0.8206
Epoch 5/100
60000/60000 [==============================] - 276s 5ms/step - loss: 0.4626 - accuracy: 0.8304 - val_loss: 0.5183 - val_accuracy: 0.8121
Epoch 6/100
60000/60000 [==============================] - 276s 5ms/step - loss: 0.4600 - accuracy: 0.8316 - val_loss: 0.5200 - val_accuracy: 0.8087
Epoch 7/100
60000/60000 [==============================]